In [1]:
import pandas as pd

tableA = [(100, 'chocolate, sprinkles'),
         (101, 'chocolate, filled'),
         (102, 'glazed')]
labels = ['product', 'tags']
dfA = pd.DataFrame.from_records(tableA, columns=labels)

tableB = [('A', 100),
           ('A', 101),
           ('B', 101),
           ('C', 100),
           ('C', 102),
           ('B', 101),
           ('A', 100),
           ('C', 102)]
labels = ['customer', 'product']
dfB = pd.DataFrame.from_records(tableB, columns=labels)

### Input
lets take a look at provided dataframes dfA & dfB

In [3]:
dfA

,product,tags
0,100,"chocolate, sprinkles"
1,101,"chocolate, filled"
2,102,glazed


In [4]:
dfB

,customer,product
0,A,100
1,A,101
2,B,101
3,C,100
4,C,102
5,B,101
6,A,100
7,C,102


### Merge
Lets see what products did a Customer buy from the transaction by merging both the tables dfA and dfB based on an inner join

In [98]:
result=pd.merge(dfA,dfB,how='inner', on='product')

   product                  tags customer
0      100  chocolate, sprinkles        A
1      100  chocolate, sprinkles        C
2      100  chocolate, sprinkles        A
3      101     chocolate, filled        A
4      101     chocolate, filled        B
5      101     chocolate, filled        B
6      102                glazed        C
7      102                glazed        C


In [100]:
result

,product,tags,customer
0,100,"chocolate, sprinkles",A
1,100,"chocolate, sprinkles",C
2,100,"chocolate, sprinkles",A
3,101,"chocolate, filled",A
4,101,"chocolate, filled",B
5,101,"chocolate, filled",B
6,102,glazed,C
7,102,glazed,C


### Split
Here we need to perform a special split columnwise into rows which can be achieved by the below code 

In [102]:
result1=(result.set_index(result.columns.drop('tags',1)
        .tolist()).tags.str.split(', ', expand=True).stack().reset_index()
        .rename(columns={0:'tags'}).loc[:, result.columns])

In [91]:
result1

,product,tags,customer
0,100,chocolate,A
1,100,sprinkles,A
2,100,chocolate,C
3,100,sprinkles,C
4,100,chocolate,A
5,100,sprinkles,A
6,101,chocolate,A
7,101,filled,A
8,101,chocolate,B
9,101,filled,B


### Pivot
Now, our table looks beatifully organized with single values in tags column, lets perform pivot using the pivot_table function to calculate the individual number of items a customer has purchased  

In [93]:
dfC=result1.pivot_table(values='tags',index=['customer'], columns=['tags'], 
                         aggfunc='size')
print(dfC)

tags      chocolate  filled  glazed  sprinkles
customer                                      
A               3.0     1.0     NaN        2.0
B               2.0     2.0     NaN        NaN
C               1.0     NaN     2.0        1.0


### Impute
Lets impute the null values by replacing NaNs with 0s as that makes sense in reality

In [120]:
dfC.fillna(0, inplace=True)

In [118]:
dfC=dfC.astype(int) #converting float value type to int

### Output

In [119]:
dfC

tags,chocolate,filled,glazed,sprinkles
customer,,,,
A,3,1,0,2
B,2,2,0,0
C,1,0,2,1
